In [1]:
from model import *
from data import *

Using TensorFlow backend.


In [ ]:
data_gen_args = dict(rotation_range=360,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(16,'./Data','image','label',data_gen_args,save_to_dir = None)

num_batch = 256
for i,batch in enumerate(myGene):
    if(i >= num_batch):
        break

Found 47 images belonging to 1 classes.
Found 47 images belonging to 1 classes.


In [ ]:
model = unet()
model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=500,epochs=20,callbacks=[model_checkpoint])

In [ ]:
import cv2,glob
import matplotlib.pyplot as plt

TARGET_SIZE = (256,256)
def preprocesss(img):

    img = trans.resize(img, TARGET_SIZE)
    img = np.reshape(img, img.shape + (1,)) if (not False) else img
    img = np.reshape(img, (1,) + img.shape)
    return img


def get_mask(width,height,npyfile):

    for i,item in enumerate(npyfile):

        img = item[:,:,0]
        img = trans.resize(img, (width, height))

        return img


def predict_unet(img,model):


    width, height = img.shape

    img = preprocesss(img)


#     with graph.as_default():
    mask = model.predict(img)


    mask = get_mask(width, height, mask)
    
    return mask

In [ ]:
model=unet(pretrained_weights='./unet.hdf5')
for filename in glob.glob("/additional_drive/FahadAli/MarkerDetection/UNET/Test/NewTest/*.*g"):
    
    print(filename)
    image = cv2.imread(filename)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    plt.imshow(image,cmap='gray')
    plt.show()
    mask = predict_unet(image,model)
    plt.imshow(mask,cmap='gray')
    plt.show()
    cv2.imwrite("/additional_drive/FahadAli/MarkerDetection/UNET/Results/" + filename,mask)
    
    
    